In [28]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv(Path(os.getcwd()).parents[1] / "OpenAI" / ".env")  # loads from .env in OpenAI directory
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=api_key)

response = client.responses.create(
    model="gpt-5.2",
    input="Write a short bedtime story about a unicorn."
)

print(response.output_text)

On a quiet hill beyond the last sleepy house, there was a small meadow that only the moon seemed to remember. The grass there grew soft as a blanket, and the flowers closed their petals at night like they were tucking themselves in.

In that meadow lived a unicorn named Luma.

Luma wasn’t the kind of unicorn who liked loud parades or shiny castles. She liked gentle things: warm breezes, slow songs of crickets, and the way the stars blinked as if they were practicing winks. Her horn glowed faintly—not bright like a lantern, but cozy like a night-light left on for comfort.

Every night, Luma took a careful walk along the edge of the woods.

Not because she was afraid—unicorns aren’t easily frightened—but because the woods could get tangled with worries. Little worries, big worries, worries that followed people home and tried to climb into bed with them.

Luma had a way of helping.

She’d pause by a fern and listen. If she heard a worry—something like *What if tomorrow is hard?* or *What 

In [39]:
INSTRUCTIONS = """
You are a parser for short French sentences (children / IME level).

Extract ONLY elements that can be illustrated with pictograms.
Never invent words not present in the sentence.
If nothing is found for a field, use null or [].

Return ONLY valid JSON (no surrounding text).
The output MUST be a JSON list, one item per input sentence, in the same order.

Each item schema:
{
  "sentence": string,
  "subject": string | null,
  "verb": { "lemma": string, "text": string } | null,
  "slots": [
    {
      "type": "object" | "color" | "weather" | "place" | "time",
      "head": string,
      "full": string
    }
  ]
}

Rules:
- Put direct objects in slots with type="object".
- Put prepositional complements like "avec X", "sous X", "à X", "dans X" in slots as:
  - type="weather" if X is a weather word (pluie, soleil, neige, vent, orage, nuage).
  - type="place" if X is a place word (école, maison, parc, télé, ville).
  - otherwise type="object".
- Put color adjectives (rose, blanc, noir, rouge, bleu, vert, jaune, gris, orange, violet) as slots with type="color".
- If a sentence has two colors ("rose et blanc"), output two color slots.

""".strip()

In [40]:
sentences = [
    "Papa a bu son café avec du sucre.",
    "Le chat noir dort avec les chatons.",
    "Mon vélo est rose et blanc.",
    "Emma a promené son chien sous la pluie."
]

resp = client.responses.create(
    model="gpt-4o-mini",
    instructions=INSTRUCTIONS,
    input="\n".join(f"- {s}" for s in sentences)
)


In [41]:
import json

raw = resp.output_text

try:
    output_json = json.loads(raw)  # to verify it's valid JSON
    print("Valid JSON")
    print(json.dumps(output_json, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("Invalid JSON:", e)
    print("Raw output:", raw)

Valid JSON
[
  {
    "sentence": "Papa a bu son café avec du sucre.",
    "subject": "Papa",
    "verb": {
      "lemma": "boire",
      "text": "a bu"
    },
    "slots": [
      {
        "type": "object",
        "head": "café",
        "full": "son café"
      },
      {
        "type": "object",
        "head": "sucre",
        "full": "du sucre"
      }
    ]
  },
  {
    "sentence": "Le chat noir dort avec les chatons.",
    "subject": "Le chat noir",
    "verb": {
      "lemma": "dormir",
      "text": "dort"
    },
    "slots": [
      {
        "type": "object",
        "head": "chatons",
        "full": "les chatons"
      }
    ]
  },
  {
    "sentence": "Mon vélo est rose et blanc.",
    "subject": "Mon vélo",
    "verb": {
      "lemma": "être",
      "text": "est"
    },
    "slots": [
      {
        "type": "color",
        "head": "rose",
        "full": "rose"
      },
      {
        "type": "color",
        "head": "blanc",
        "full": "blanc"
      }
    ]
  }

In [ ]:
sentences = [
    "Papa a bu son café avec du sucre.",
    "Je joue à cache-cache avec la voisine.",
    "Mon vélo est rose et blanc.",
    "Tata m’a acheté un casque rose pour ma fête.",
    "Le chat dort avec les chatons.",
    "Elle a bu du jus de pommes.",
    "Maman a décoré le mur pour la fête.",
    "Le samedi papa tire à l’arc.",
    "Le bébé koala a vu sa maman.",
    "Je regarde Dora à la télé chaque samedi.",
]

results = []
for i, s in enumerate(sentences, start=1):
    resp = client.responses.create(
        model="gpt-4o-mini",
        instructions=INSTRUCTIONS,
        input=s,
    )
    raw = resp.output_text
    try:
        parsed = json.loads(raw)
        ok = True
    except Exception as e:
        parsed = {"_error": str(e), "_raw": raw}
        ok = False

    results.append({"i": i, "sentence": s, "ok": ok, "parsed": parsed})

# Affichage compact
for r in results:
    print(f"\n[{r['i']}] ok={r['ok']}  {r['sentence']}")
    print(r["parsed"])